<a href="https://colab.research.google.com/github/juliewang2020/FreeRealEstate/blob/master/parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Data


In [0]:
import pandas as pd
import io
df = pd.read_csv(io.BytesIO(uploaded['data.csv']))

#get rid of ID and CreatedDate
features = df.iloc[:, 1:]
features.drop('CreatedDate', axis = 1, inplace=True)


In [0]:
############## Cleaning the HTML

import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

for i in range(0, 1933):
  body_text = features.iloc[i , 2]
  body_text = cleanhtml(body_text)
  body_text = body_text.replace('\n', ' ')
  features.loc[i, 'Body'] = body_text
 

### HI CALISTA

In [0]:
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types